In [1090]:
# Import and read dataset
import pandas as pd

# Dataset prepro 
from operator import is_not
from functools import partial

# Tokenization
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer
import numpy as np

# To sort Dataset
from operator import itemgetter, attrgetter

# Train NER from a blank spacy model
import spacy

# Score the result
from spacy.scorer import Scorer
from spacy.gold import GoldParse

In [1091]:
# Dataset path
entities = 'assignment 1 data/chemprot_training/chemprot_training_entities'
abstracts = 'assignment 1 data/chemprot_training/chemprot_training_abstracts'
entities_t = 'assignment 1 data/chemprot_test/chemprot_test_entities'
abstracts_t = 'assignment 1 data/chemprot_test/chemprot_test_abstracts'

In [1092]:
# Import dataset
train_entities = pd.read_csv(entities + '.tsv', sep='\t', header=None, names=["PMID", "ENTITY", "TYPE", "START", "END", "TEXT"])
train_abstracts = pd.read_csv(abstracts + '.tsv', sep='\t', header=None, names=["PMID", "TITLE", "ABSTRACT"])

test_entities = pd.read_csv(entities_t + '.tsv', sep='\t', header=None, names=["PMID", "ENTITY", "TYPE", "START", "END", "TEXT"])
test_abstracts = pd.read_csv(abstracts_t + '.tsv', sep='\t', header=None, names=["PMID", "TITLE", "ABSTRACT"])

In [1093]:
# NA is treated as NaN, so covert NaN to 'NA'
train_entities = train_entities.replace(np.nan, 'NA', regex=True)
test_entities = test_entities.replace(np.nan, 'NA', regex=True)

In [1094]:
# Data Preprocess
def data_prepro(entity, abstract):
    """
    Data Preprocess
    """
    entity = entity.replace("GENE-N", "GENE")
    entity = entity.replace("GENE-Y", "GENE")
    
    sentence = pd.DataFrame(columns=["PMID", "SENTENCE", "START", "END"])
    result = []
    
    
    for i in range(len(abstract)):
        id = abstract.PMID[i]
        id_str = str(id)
        title = abstract.TITLE[i]
        abst = abstract.ABSTRACT[i]
        total = title + " " + abst

        """
        Tokenize sentences
        """
        sentence_data = total
        for start, end in PunktSentenceTokenizer().span_tokenize(sentence_data):
            df = pd.DataFrame(np.array([[id ,sentence_data[start:end], start, end]]), columns=["PMID", "SENTENCE", "START", "END"])
            sentence = pd.concat([sentence, df], ignore_index=True)

        
        sent = sentence[sentence['PMID']==id_str]
        sent = sent.values.tolist()
        
          
        data = entity.loc[entity['PMID'] == id]
        data_t = data.values.tolist()
        
        l1 = []
        l2 = []
        """
        Check entities which are overlapping with each other
        And Remove longer ones
        """
        for k in range(len(sent)):
        
                   
            for l in range(len(data_t)):
                
                b = l + 1
                if len(data_t)>1:
                            
                        data_t.sort(reverse=False, key=itemgetter(3))
                            
                        for c in range(b,len(data_t)):
                                
                            if ((int(data_t[l][3]) <= int(data_t[c][3])) 
                                   and (int(data_t[c][3]) < int(data_t[l][4]))):
                                data_t[c][0] = None
            
            """
            Connect the tokenized sentences and entities
            Convert dataset into 'Spacy' form
            """
            l3 = [] 
            for j in range(len(data_t)):
                
                start = int(data_t[j][3]- int(sent[k][2]))
                end = int(data_t[j][4]- int(sent[k][2]))
                text = str(data_t[j][5])
                sen = sent[k][1]
                text_sent = sen[start:end]
                        
                 
                if ((int(sent[k][2]) <= int(data_t[j][3])) 
                    and (int(data_t[j][3]) < int(sent[k][3]))):
                    
                    """
                    Check if text and sentece do not fit
                    """    
                    if text == text_sent:
                        
                        if data_t[j][0] != None:
                            l3.append((start, end, data_t[j][2]))
                 
                     
                else:
                    j = j + 1
            
            l1.append(sent[k][1])
            l2.append({"entities":l3})

        TRAIN = list(zip(l1,l2))
        result.extend(TRAIN)
        
    """
    Remove the empty entities (None, "")
    """
    for m in range(len(result)):
            if result[m][1]["entities"] == [] or result[m][1]['entities'][0] == '':
                result[m] = None

    result = list(filter(partial(is_not, None), result))
    
        
                
    return result

In [1095]:
#Data Preprossess
TRAIN_DATA = data_prepro(train_entities, train_abstracts)
TEST_DATA = data_prepro(test_entities, test_abstracts)

In [1096]:
# Train model with blank Spacy model

nlp = spacy.blank("en")
nlp.add_pipe(nlp.create_pipe('ner'))
nlp.begin_training()

# The pipeline component NER
ner=nlp.get_pipe("ner")

In [1079]:
# Train Model with pre-existing spacy model

nlp=spacy.load("en_core_web_sm") 

# The pipeline component NER
ner=nlp.get_pipe('ner')

In [1097]:
# New labels
LABEL1 = "GENE"
LABEL2 = "CHEMICAL"

In [1098]:
# Add the new labels to NER
ner.add_label(LABEL1)
ner.add_label(LABEL2)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# Pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [1120]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

    sizes = compounding(4.0, 32.0, 1.001)
      # Training for 30 iterations     
    for itn in range(30):
        # shuffle examples before training
        random.shuffle(TRAIN_DATA)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=sizes)
        # ictionary to store losses
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
          # Calling update() over the iteration
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

Losses {'ner': 36.34952199459076}
Losses {'ner': 68.01470136642456}
Losses {'ner': 99.0376123055612}
Losses {'ner': 127.64468984452833}
Losses {'ner': 162.39533483384002}
Losses {'ner': 195.1953314482962}
Losses {'ner': 229.14405482409347}
Losses {'ner': 257.43399526512076}
Losses {'ner': 281.4840738359635}
Losses {'ner': 328.30721631979395}
Losses {'ner': 375.28330639337946}
Losses {'ner': 398.55451253389765}
Losses {'ner': 426.06826079027087}
Losses {'ner': 451.58423304693133}
Losses {'ner': 479.8567149652663}
Losses {'ner': 505.14520469855563}
Losses {'ner': 535.6371121783501}
Losses {'ner': 560.5360994566923}
Losses {'ner': 596.9958530296332}
Losses {'ner': 631.3538032402045}
Losses {'ner': 668.0293217529303}
Losses {'ner': 714.3064246558403}
Losses {'ner': 734.638851753675}
Losses {'ner': 747.9156429282084}
Losses {'ner': 797.9993777385607}
Losses {'ner': 837.4267120840675}
Losses {'ner': 861.3693904259911}
Losses {'ner': 887.0966003266525}
Losses {'ner': 908.0154660073471}
Losses

Losses {'ner': 7762.880260918285}
Losses {'ner': 7786.685790155078}
Losses {'ner': 7816.552252147342}
Losses {'ner': 7843.1700876974455}
Losses {'ner': 7862.926478745877}
Losses {'ner': 7904.543185831145}
Losses {'ner': 7933.573308109869}
Losses {'ner': 7966.712677120794}
Losses {'ner': 8003.680727815236}
Losses {'ner': 8054.092821752633}
Losses {'ner': 8098.161380982007}
Losses {'ner': 8131.421659624185}
Losses {'ner': 8157.215437639107}
Losses {'ner': 8185.094417804975}
Losses {'ner': 8236.415956966266}
Losses {'ner': 8277.782315783523}
Losses {'ner': 8302.543097548507}
Losses {'ner': 8329.137816243194}
Losses {'ner': 8367.610442349076}
Losses {'ner': 8401.427127013303}
Losses {'ner': 8438.814142057992}
Losses {'ner': 8467.25233782301}
Losses {'ner': 8499.07619763384}
Losses {'ner': 8546.481080412254}
Losses {'ner': 8595.18685811697}
Losses {'ner': 8623.53660505949}
Losses {'ner': 8661.164338528499}
Losses {'ner': 8705.568900308892}
Losses {'ner': 8744.147668097954}
Losses {'ner': 88

Losses {'ner': 18078.301681596855}
Losses {'ner': 18121.943241555313}
Losses {'ner': 18165.016777534107}
Losses {'ner': 18212.18261326484}
Losses {'ner': 18257.377951037208}
Losses {'ner': 18285.46335421882}
Losses {'ner': 18337.000627148966}
Losses {'ner': 18379.40580450378}
Losses {'ner': 18433.71467303119}
Losses {'ner': 18475.460880268434}
Losses {'ner': 18513.33857070289}
Losses {'ner': 18564.170809377054}
Losses {'ner': 18617.45084678016}
Losses {'ner': 18650.69542594912}
Losses {'ner': 18716.761950156713}
Losses {'ner': 18778.453575751806}
Losses {'ner': 18811.235105774427}
Losses {'ner': 18874.13728310826}
Losses {'ner': 18900.013532183195}
Losses {'ner': 18944.074516556288}
Losses {'ner': 18987.985262057806}
Losses {'ner': 19038.0399494982}
Losses {'ner': 19062.693062744165}
Losses {'ner': 19103.213344416643}
Losses {'ner': 19142.21688874724}
Losses {'ner': 19201.064425311113}
Losses {'ner': 19242.692834219957}
Losses {'ner': 19292.875555298353}
Losses {'ner': 19335.1795898937

Losses {'ner': 30767.4985420468}
Losses {'ner': 30825.961101269975}
Losses {'ner': 30886.87007710959}
Losses {'ner': 30955.669126010194}
Losses {'ner': 31032.210910535112}
Losses {'ner': 31080.55184028174}
Losses {'ner': 31133.903913474336}
Losses {'ner': 31222.20926330115}
Losses {'ner': 31301.302732444063}
Losses {'ner': 31364.56340096022}
Losses {'ner': 31407.584863400712}
Losses {'ner': 31448.00142208363}
Losses {'ner': 31519.165379441038}
Losses {'ner': 31581.624961531415}
Losses {'ner': 31676.09494830591}
Losses {'ner': 31733.795389666382}
Losses {'ner': 31810.610646500412}
Losses {'ner': 31892.733448042694}
Losses {'ner': 31939.142303719345}
Losses {'ner': 32030.086579813782}
Losses {'ner': 32085.273738517586}
Losses {'ner': 32165.875468864266}
Losses {'ner': 32243.123188152138}
Losses {'ner': 32300.68275631887}
Losses {'ner': 32360.38548053724}
Losses {'ner': 32453.64228452665}
Losses {'ner': 32498.44642640573}
Losses {'ner': 32579.227205767456}
Losses {'ner': 32646.15950085145

Losses {'ner': 47663.81961452424}
Losses {'ner': 47730.7069641298}
Losses {'ner': 47798.16912376344}
Losses {'ner': 47883.1534110254}
Losses {'ner': 47961.811156744356}
Losses {'ner': 48049.56767773091}
Losses {'ner': 48127.391058916444}
Losses {'ner': 48203.99083375394}
Losses {'ner': 48297.01502751767}
Losses {'ner': 48362.85935997426}
Losses {'ner': 48452.28098034322}
Losses {'ner': 48548.5239632076}
Losses {'ner': 48621.276884788866}
Losses {'ner': 48699.84776997029}
Losses {'ner': 48775.87475346982}
Losses {'ner': 48825.22203874051}
Losses {'ner': 48901.744596893186}
Losses {'ner': 48981.337842041845}
Losses {'ner': 49075.27040582358}
Losses {'ner': 49207.34305339515}
Losses {'ner': 49273.97099261939}
Losses {'ner': 49346.26744132697}
Losses {'ner': 49430.815856034154}
Losses {'ner': 49517.511243874425}
Losses {'ner': 49607.60768848121}
Losses {'ner': 49673.21045511424}
Losses {'ner': 49763.545193607206}
Losses {'ner': 49851.20797174632}
Losses {'ner': 49920.63583510577}
Losses {'

...

Losses {'ner': 288.21753311157227}
Losses {'ner': 561.3628826141357}
Losses {'ner': 821.007568359375}
Losses {'ner': 1082.7735767364502}
Losses {'ner': 1327.5246028900146}
Losses {'ner': 1600.9219989776611}
Losses {'ner': 1867.408151626587}
Losses {'ner': 2165.9621238708496}
Losses {'ner': 2432.318862915039}
Losses {'ner': 2664.835023880005}
Losses {'ner': 2897.4168157577515}
Losses {'ner': 3119.8955640792847}
Losses {'ner': 3362.5851678848267}
Losses {'ner': 3593.1814227104187}
Losses {'ner': 3815.2422556877136}
Losses {'ner': 4085.964334011078}
Losses {'ner': 4340.024636745453}
Losses {'ner': 4562.934617519379}
Losses {'ner': 4799.675280094147}
Losses {'ner': 5053.083586215973}
Losses {'ner': 5280.812326908112}
Losses {'ner': 5502.788485050201}
Losses {'ner': 5737.742512226105}
Losses {'ner': 6008.24306344986}
Losses {'ner': 6262.794984340668}
Losses {'ner': 6507.874825954437}
Losses {'ner': 6745.464061260223}
Losses {'ner': 6973.306435108185}
Losses {'ner': 7259.2456822395325}
Losse

Losses {'ner': 60089.589587688446}
Losses {'ner': 60325.11783361435}
Losses {'ner': 60554.524550914764}
Losses {'ner': 60774.54354143143}
Losses {'ner': 61011.24287080765}
Losses {'ner': 61224.868725299835}
Losses {'ner': 61483.66488409042}
Losses {'ner': 61781.49689245224}
Losses {'ner': 62052.017673015594}
Losses {'ner': 62328.38474941254}
Losses {'ner': 62578.963872909546}
Losses {'ner': 62806.794969558716}
Losses {'ner': 63052.37008571625}
Losses {'ner': 63328.52518749237}
Losses {'ner': 63575.27959728241}
Losses {'ner': 63898.02690792084}
Losses {'ner': 64124.01953792572}
Losses {'ner': 64381.95479297638}
Losses {'ner': 64664.413659095764}
Losses {'ner': 64913.269658088684}
Losses {'ner': 65184.3061876297}
Losses {'ner': 65447.310826301575}
Losses {'ner': 65693.4940443039}
Losses {'ner': 65989.42427158356}
Losses {'ner': 66248.4532327652}
Losses {'ner': 66504.62596607208}
Losses {'ner': 66583.8663213253}
Losses {'ner': 228.6183319091797}
Losses {'ner': 473.81575775146484}
Losses {

Losses {'ner': 53182.08949661255}
Losses {'ner': 53399.86420917511}
Losses {'ner': 53618.57653903961}
Losses {'ner': 53838.407660484314}
Losses {'ner': 54082.988463401794}
Losses {'ner': 54326.78840351105}
Losses {'ner': 54568.9931306839}
Losses {'ner': 54807.06999492645}
Losses {'ner': 55070.29441547394}
Losses {'ner': 55281.320300102234}
Losses {'ner': 55505.44826602936}
Losses {'ner': 55766.65706348419}
Losses {'ner': 55991.93905162811}
Losses {'ner': 56222.28339290619}
Losses {'ner': 56479.85607624054}
Losses {'ner': 56790.896887779236}
Losses {'ner': 57059.60706996918}
Losses {'ner': 57298.763392448425}
Losses {'ner': 57533.81739330292}
Losses {'ner': 57807.60699558258}
Losses {'ner': 58089.42829990387}
Losses {'ner': 58398.15506839752}
Losses {'ner': 58643.07055950165}
Losses {'ner': 58867.70521450043}
Losses {'ner': 59053.330843925476}
Losses {'ner': 59347.592871665955}
Losses {'ner': 59559.24497509003}
Losses {'ner': 59802.93943595886}
Losses {'ner': 60051.12334442139}
Losses {

Losses {'ner': 47330.846002578735}
Losses {'ner': 47564.130021095276}
Losses {'ner': 47845.20851421356}
Losses {'ner': 48123.85523700714}
Losses {'ner': 48437.923338890076}
Losses {'ner': 48692.986313819885}
Losses {'ner': 48922.41887760162}
Losses {'ner': 49209.372599601746}
Losses {'ner': 49468.69521808624}
Losses {'ner': 49693.70984363556}
Losses {'ner': 49924.46021747589}
Losses {'ner': 50165.100043296814}
Losses {'ner': 50424.35764980316}
Losses {'ner': 50670.731341362}
Losses {'ner': 50877.243094444275}
Losses {'ner': 51125.16407394409}
Losses {'ner': 51394.920108795166}
Losses {'ner': 51661.881063461304}
Losses {'ner': 51893.8653049469}
Losses {'ner': 52128.412631988525}
Losses {'ner': 52348.84334373474}
Losses {'ner': 52622.26388549805}
Losses {'ner': 52866.36665344238}
Losses {'ner': 53066.41647529602}
Losses {'ner': 53345.42152881622}
Losses {'ner': 53596.40288448334}
Losses {'ner': 53833.28974056244}
Losses {'ner': 54041.2967672348}
Losses {'ner': 54267.70438289642}
Losses {

Losses {'ner': 40393.05429935455}
Losses {'ner': 40696.36341190338}
Losses {'ner': 40964.59169101715}
Losses {'ner': 41173.94366931915}
Losses {'ner': 41438.46305370331}
Losses {'ner': 41726.89014339447}
Losses {'ner': 41998.58050823212}
Losses {'ner': 42227.66224384308}
Losses {'ner': 42485.672947883606}
Losses {'ner': 42730.71054458618}
Losses {'ner': 42993.29711723328}
Losses {'ner': 43233.59755039215}
Losses {'ner': 43514.81617832184}
Losses {'ner': 43760.73791408539}
Losses {'ner': 44002.53793811798}
Losses {'ner': 44264.68508434296}
Losses {'ner': 44527.88319683075}
Losses {'ner': 44792.71981334686}
Losses {'ner': 44999.62142467499}
Losses {'ner': 45258.31857585907}
Losses {'ner': 45525.74799442291}
Losses {'ner': 45772.24465084076}
Losses {'ner': 46004.11283779144}
Losses {'ner': 46252.53097629547}
Losses {'ner': 46486.84146404266}
Losses {'ner': 46746.96782588959}
Losses {'ner': 46982.178793907166}
Losses {'ner': 47205.24756717682}
Losses {'ner': 47448.66894054413}
Losses {'ner

In [1121]:
# Testing the model
doc = nlp("The fusion protein abatacept is a selective costimulation modulator that avidly binds to the CD80/CD86 ligands on an antigen-presenting cell, resulting in the inability of these ligands to engage the CD28 receptor on the T cell.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('CD80', 'GENE'), ('CD86', 'GENE'), ('CD28 receptor', 'GENE')]


In [1122]:
# Evaluater
def evaluate(ner_model, examples):
    scorer = Scorer()

    for input, annot in examples:
        doc_gold_text = ner_model.make_doc(input)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores

In [1123]:
results_train = evaluate(nlp, TRAIN_DATA)

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "(R)-Ketoprofen (1) was previously reported to be a..." with entities "[(0, 14, 'CHEMICAL'), (99, 104, 'GENE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The keto and phenolic -OH are major factors that a..." with entities "[(4, 8, 'CHEMICAL'), (13, 21, 'CHEMICAL'), (23, 25...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "After CUI-3I

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The development of specific somatostatin subtypes ..." with entities "[(28, 40, 'CHEMICAL'), (70, 75, 'GENE'), (82, 87, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "In the present study, we determined the dose-respo..." with entities "[(65, 76, 'CHEMICAL'), (81, 92, 'CHEMICAL'), (105,...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Structure-activity relationship studies of these d..." with entities "[(121, 125, 'GENE'), (126, 131, 'GENE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The PGE1-sensitive component of the current was ma..." with entities "[(4, 8, 'CHEMICAL'), (132, 142, 'CHEMICAL')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "5'-flanking region of

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Moreover, S1611 and S3226, both specific inhibitor..." with entities "[(10, 15, 'CHEMICAL'), (20, 25, 'CHEMICAL'), (55, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "By using the whole-cell patch-clamp configurations..." with entities "[(61, 64, 'CHEMICAL'), (125, 130, 'CHEMICAL'), (14...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Since the LQT syndrome mutant KvLQT1-G306R suppres..." with entities "[(30, 36, 'GENE'), (37, 42, 'GENE'), (103, 109, 'G...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Epi increased the activity of the human WNT6 promo..." with entities "[(0, 3, 'CHEMICAL'), (34, 53, 'GENE'), (62, 66, 'G...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

...

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Dandelion leaf extract protects against liver inju..." with entities "[(64, 74, 'CHEMICAL'), (80, 87, 'CHEMICAL')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Accordingly, PTN-/- and MK-/- mice show an increas..." with entities "[(13, 16, 'GENE'), (24, 26, 'GENE'), (70, 81, 'CHE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Inhibiti

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Toxicity studies show that the organotypic culture..." with entities "[(74, 87, 'CHEMICAL'), (93, 106, 'CHEMICAL'), (146...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Without agonist stimulation, the ECL2 in the gain ..." with entities "[(69, 74, 'GENE'), (113, 118, 'GENE'), (135, 139, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "This work resulted in the discovery of the 5-morph..." with entities "[(43, 83, 'CHEMICAL'), (146, 150, 'GENE'), (193, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "[Trp(7)]GnRH-I and [Trp(8)]GnRH-I but not [His(5)]..." with entities "[(8, 14, 'GENE'), (27, 33, 'GENE'), (50, 56, 'GENE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "CONCLUSION: Albiglutide is the fourth GLP-1 RA app..." with entities "[(38, 45, 'GENE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Furthermore, the expression of these GnRHs was cor..." with entities "[(37, 42, 'CHEMICAL'), (81, 92, 'GENE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The number of times DEX-P was administered was 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Cytochrome P450-mediated herb-drug interaction pot..." with entities "[(0, 15, 'GENE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "We hypothesize that L0 interacts with the K(IR) N ..." with entities "[(42, 47, 'GENE'), (48, 49, 'CHEMICAL'), (79, 80, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Our study found that oral administr

In [1124]:
results_test = evaluate(nlp, TEST_DATA)

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "In vivo, the treatment groups with TFs (100, 200, ..." with entities "[(97, 112, 'CHEMICAL'), (124, 144, 'GENE'), (149, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Synaptotagmin 1 is required for vesicular Ca(2+) /..." with entities "[(0, 15, 'GENE'), (42, 48, 'CHEMICAL'), (50, 54, '...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "GDAP1-induced peroxisomal fission is dependent on ..." with entities "[(0, 5, 'GENE'), (71, 91, 'GENE'), (100, 104, 'GEN...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "New targeted therapies include 5-HT(1F) receptor a..." with entities "[(31, 35, 'CHEMICAL'), (59, 90, 'GENE'), (92, 96, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Identification of a RhoA- and SRF-Dependent Mechan..." with entities "[(20, 24, 'GENE'), (30, 33, 'GENE'), (57, 65, 'CHE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Exploration of the molecular events that underlie ..." with entities "[(50, 58, 'CHEMICAL'), (77, 79, 'GENE'), (102, 106...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "5 Channel blockade by cisapride was modulated by [..." with entities "[(22, 31, 'CHEMICAL'), (50, 52, 'CHEMICAL'), (118,...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "A general islet-cell dysfunction including insulin..." with entities "[(43, 50, 'GENE'), (56, 64, 'GENE')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Increased Cat3-m

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Involvement of interleukin-6-regulated nitric oxid..." with entities "[(15, 28, 'GENE'), (39, 51, 'CHEMICAL'), (144, 152...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "IL-6-neutralizing antibodies effectively inhibited..." with entities "[(0, 4, 'GENE'), (55, 63, 'CHEMICAL')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The increased 

...

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Pre-treatment with an antibody raised against the ..." with entities "[(50, 58, 'GENE'), (160, 163, 'CHEMICAL'), (167, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Death was dependent, at least in part, on the stim..." with entities "[(65, 69, 'CHEMICAL')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "MicroPET imaging in nonhuman p

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Inhibition studies of (-)-huperzine A with human b..." with entities "[(22, 37, 'CHEMICAL'), (43, 70, 'GENE'), (160, 188...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The smooth muscle relaxant, W-7, which is believed..." with entities "[(28, 31, 'CHEMICAL'), (89, 95, 'CHEMICAL'), (96, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "B-domain-deleted (BDD)-FVIII yields higher mRNA le..." with entities "[(0, 8, 'GENE'), (23, 28, 'GENE'), (96, 105, 'GENE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Selective COX-2 inhibitors (COXib) belonging to th..." with entities "[(10, 15, 'GENE'), (28, 31, 'GENE'), (61, 80, 'CHE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The AGE-induced increases in VEGF expression and P..." with entities "[(29, 33, 'GENE'), (49, 52, 'GENE'), (99, 102, 'GE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Apart from the parent compound two metabolites wer..." with entities "[(70, 79, 'CHEMICAL'), (80, 90, 'CHEMICAL'), (92, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "Neuronal AR regulates hypothalamic insulin signali..." with entities "[(9, 11, 'GENE'), (35, 42, 'GENE'), (67, 84, 'GENE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text "The inhibitory effects of icariin on PDE5 and PDE4..." with entities "[(26, 33, 'CHEMICAL'), (37, 41, 'GENE'), (46, 50, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  
/Users/song92/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W030] Some entities could not be aligned in the text 

In [1125]:
precis = results_train['ents_p'] 
recall = results_train['ents_r']
f_score = results_train['ents_f']
f_score_gene = results_train['ents_per_type']['GENE']['f']
f_score_chem = results_train['ents_per_type']['CHEMICAL']['f']

print("Model Precision",'\n>>>', precis,'\n')
print("Model Recall ",'\n>>>', recall ,'\n')
print("Model F Score ",'\n>>>', f_score,'\n')
print("Label: GENE F Score ",'\n>>>', f_score_gene,'\n')
print("Label: CHEMICAL F Score ",'\n>>>', f_score_chem,'\n')

Model Precision 
>>> 99.19924188580906 

Model Recall  
>>> 99.64778676820562 

Model F Score  
>>> 99.42300842930074 

Label: GENE F Score  
>>> 99.3460764587525 

Label: CHEMICAL F Score  
>>> 99.49179221947381 



In [1126]:
precis = results_test['ents_p'] 
recall = results_test['ents_r']
f_score = results_test['ents_f']
f_score_gene = results_test['ents_per_type']['GENE']['f']
f_score_chem = results_test['ents_per_type']['CHEMICAL']['f']

print("Model Precision",'\n>>>', precis,'\n')
print("Model Recall ",'\n>>>', recall ,'\n')
print("Model F Score ",'\n>>>', f_score,'\n')
print("Label: GENE F Score ",'\n>>>', f_score_gene,'\n')
print("Label: CHEMICAL F Score ",'\n>>>', f_score_chem,'\n')

Model Precision 
>>> 77.61965268953833 

Model Recall  
>>> 75.0850165936002 

Model F Score  
>>> 76.33129933148676 

Label: GENE F Score  
>>> 74.32402693622453 

Label: CHEMICAL F Score  
>>> 78.13414152062195 



In [ ]:
# used for tokenization
https://stackoverflow.com/questions/14776317/finding-exact-position-of-tokenized-sentences
# used for modeling
https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/
# used for scorer
https://stackoverflow.com/questions/44827930/evaluation-in-a-spacy-ner-model